In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score, log_loss
from sklearn.preprocessing import MinMaxScaler

In [2]:
df = pd.read_csv("C:/Training/Academy/Statistics (Python)/Cases/human-resources-analytics/HR_comma_sep.csv")
dum_df = pd.get_dummies(df,drop_first=True)
dum_df.head()

satisfaction_level  last_evaluation  number_project  average_montly_hours  \
0                0.38             0.53               2                   157   
1                0.80             0.86               5                   262   
2                0.10             0.77               6                   247   
3                0.92             0.85               5                   259   
4                0.89             1.00               5                   224   

   time_spend_company  Work_accident  left  promotion_last_5years  \
0                   3              0     1                      0   
1                   6              0     1                      0   
2                   4              0     1                      0   
3                   5              0     1                      0   
4                   5              0     1                      0   

   Department_RandD  Department_accounting  Department_hr  \
0             False                  False          False   
1             False                  False          False   
2             False                  False          False   
3             False                  False          False   
4             False                  False          False   

   Department_management  Department_marketing  Department_product_mng  \
0                  False                 False                   False   
1                  False                 False                   False   
2                  False                 False                   False   
3                  False                 False                   False   
4                  False                 False                   False   

   Department_sales  Department_support  Department_technical  salary_low  \
0              True               False                 False        True   
1              True               False                 False       False   
2              True               False                 False        True   
3              True               False                 False        True   
4              True               False                 False        True   

   salary_medium  
0          False  
1           True  
2          False  
3          False  
4          False

In [3]:
X = dum_df.drop('left', axis=1)
scaler = MinMaxScaler()

y = dum_df['left'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, 
                                                    random_state=23,stratify=y)
X_scl_trn = scaler.fit_transform(X_train) 
X_scl_tst = scaler.transform(X_test) 

In [4]:
X_torch = torch.from_numpy(X_scl_trn)
y_torch = torch.from_numpy(y_train)
print(X_torch.size())
print(y_torch.size())

torch.Size([10496, 18])
torch.Size([10496])


In [5]:
type(X_torch)

torch.Tensor

In [6]:
X_scl_trn.shape[1]

18

In [7]:
torch.manual_seed(23)
# Create a model
model = nn.Sequential(nn.Linear(in_features=X_scl_trn.shape[1], out_features=15),
                      nn.ReLU(),
                      nn.Linear(15, 8 ),
                      nn.ReLU(),
                      nn.Linear(8, 5),
                      nn.ReLU(),
                      nn.Linear(5,1))

In [8]:
model.parameters

<bound method Module.parameters of Sequential(
  (0): Linear(in_features=18, out_features=15, bias=True)
  (1): ReLU()
  (2): Linear(in_features=15, out_features=8, bias=True)
  (3): ReLU()
  (4): Linear(in_features=8, out_features=5, bias=True)
  (5): ReLU()
  (6): Linear(in_features=5, out_features=1, bias=True)
)>

In [9]:
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.001)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.001
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)

In [10]:
y_torch.size()

torch.Size([10496])

Prediction with initialized weights

In [11]:
y_pred = model(X_torch.float())
y_torch = y_torch.unsqueeze(1)
print(y_torch.shape)
print(y_pred.shape)

torch.Size([10496, 1])
torch.Size([10496, 1])


In [12]:
y_pred[:5]

tensor([[0.5212],
        [0.5220],
        [0.5230],
        [0.5213],
        [0.5221]], grad_fn=<SliceBackward0>)

### Initial Log Loss

In [13]:
loss = criterion(y_pred, y_torch.float())
loss

tensor(0.8638, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [14]:
for epoch in np.arange(0,1000):
       # Forward pass: Compute predicted y by passing x to the model
       y_pred_prob = model(X_torch.float())

       # Compute and print loss
       loss = criterion(y_pred_prob, y_torch.float())
       if epoch%100 == 0:
          print('epoch: ', epoch+1,' loss: ', loss.item())

       # Zero gradients, perform a backward pass, and update the weights.
       optimizer.zero_grad()

       # perform a backward pass (backpropagation)
       loss.backward()

       # Update the parameters
       optimizer.step()
#print('epoch: ', epoch+1,' loss: ', loss.item())

epoch:  1  loss:  0.8638071417808533
epoch:  101  loss:  0.845905601978302
epoch:  201  loss:  0.8292140364646912
epoch:  301  loss:  0.8136281371116638
epoch:  401  loss:  0.7990565299987793
epoch:  501  loss:  0.7854172587394714
epoch:  601  loss:  0.7726370096206665
epoch:  701  loss:  0.7606497406959534
epoch:  801  loss:  0.7493940591812134
epoch:  901  loss:  0.7388163208961487


### Training Set Log Loss

In [15]:
loss

tensor(0.7290, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [16]:
X_torch_test = torch.from_numpy(X_scl_tst)
type(X_torch_test)

torch.Tensor

In [17]:
y_pred_prob = model(X_torch_test.float()) # Equivalent predict_proba / predict
type(y_pred_prob)

torch.Tensor

In [19]:
y_pred_prob = nn.Sigmoid()(y_pred_prob)
y_pred_prob

tensor([[0.5318],
        [0.5320],
        [0.5321],
        ...,
        [0.5323],
        [0.5324],
        [0.5323]], grad_fn=<SigmoidBackward0>)

In [20]:
y_pred_prob = y_pred_prob.detach().numpy()
type(y_pred_prob)

numpy.ndarray

In [21]:
y_pred_prob.shape

(4499, 1)

In [22]:
y_pred_prob = y_pred_prob.reshape(y_test.shape[0],)
y_pred_prob.shape

(4499,)

In [23]:
y_pred = np.where(y_pred_prob >= 0.5,1,0)

y_pred[:5]

array([1, 1, 1, 1, 1])

### Test Set Accuracy Score

In [24]:
print(accuracy_score(y_test,y_pred))

0.23783062902867305


### Test Set Log Loss

In [25]:
log_loss(y_test, y_pred_prob)

0.7288702860374321